# Final Project 
================================================================================================================

**TEAM**: Star Li, Jacky Zhang, Stefan Li

## Setup

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid", 
        color_codes = True,
        font_scale = 1.5)

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LassoCV, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

### Loading in the Data

In [ ]:
data_state = pd.read_csv('final_proj_data/covid19/4.18states.csv')
data_counties = pd.read_csv('final_proj_data/covid19/abridged_couties.csv')
data_time_conf = pd.read_csv('final_proj_data/covid19/time_series_covid19_confirmed_US.csv')
data_time_de = pd.read_csv('final_proj_data/covid19/time_series_covid19_deaths_US.csv')

## Data Cleaning

In [ ]:
# - Based on our fist look at the data as well as the objective of our project, 
# we decided to work with only the U.S. data since we have the most specific data 
# regarding all the cases in the U.S. 
data_state = data_state.loc[data_state['Country_Region'] == 'US']
data_state = data_state.rename(columns={"Long_": "Long"})

data_state.head(5)

### 1. "4.18state"

In [ ]:
# fist divide the data into edge case (e.g. Diamond Princess that does not have lat and long) and general cases

edge_case = data_state[data_state['Lat'].isnull()]
edge_case.head(10)

In [ ]:
# Notice that there are only three cases in the edge_case category, and those are very specially cases 
# such as "Diamond Princess" and "Grand Princess", and the are clearly not a good representation of regional cases.
#So, we can safely drop these columns from our main focuse

general_state = data_state[data_state['Lat'].notnull()]

#### Cleaning up NaN values

In [ ]:
general_state[general_state['Recovered'].isnull()].head(5)

- Notice that there are some NaN's columns such as "Recovered", "Active", "Mortality_rate", and "Hospitalization_Rate"
- We know that, based on the provided README file, Active cases = total confirmed - total recovered - total deaths, we can safely fill out all the NaN's in Deaths, Recorvered, and Active with 0, since they are all mutually exclusive.

In [ ]:
general_state[['Recovered']] = general_state[['Recovered']].fillna(0)
general_state[['Active']] = general_state[['Active']].fillna(0)
general_state[['Mortality_Rate']] = general_state[['Mortality_Rate']].fillna(0)

# Notice that US Hospitalization Rate (%): = Total number hospitalized / Number confirmed cases,
# so if the number of People_Hospitalized is NaN or 0, 
# we can logically fill in 0 for all the NaN's in these two columns

general_state[['People_Hospitalized']] = general_state[['People_Hospitalized']].fillna(0)
general_state[['Hospitalization_Rate']] = general_state[['Hospitalization_Rate']].fillna(0)

general_state.isna().sum()

In [ ]:
#Up to this point, we see that there is only one NaN value left in the column of Last_Update

general_state[general_state['Last_Update'].isnull()]

In [ ]:
# Since "American Samoa" has such a small number of cases, it will have relatively small effect on our prediction, 
# so we decided to remove this area to keep the consistency of our dataframe.

general_state = general_state[general_state['Last_Update'].notnull()]

### 2. Joining "abridged_couties" with "time_series_covid19_confirmed_US"

- abridged_counties contains wonderful information about population health condition as well as population demograpics, and time_series_covid19_confirmed_US has a wonderful pattern of the times series of confirms in the U.S., so it would be a lot easier to select feature from this join table rather than subseting data separately from each data set when selecting features.
- Notice that we are not cleaning up the data for the death timeserise, as our main objective is to predict the confirmed cases of the next day.

In [ ]:
#- We first filtered out the territories of the U.S. as we think it 
# is not the best representation of the cases of all the other 50 major U.S. States

data_time_conf = data_time_conf.iloc[5:, :]
data_counties = data_counties.iloc[:-2, :]

data_counties.head()

- There are two rows with all NaN's at the end, we the code above dropped them.
- We decided to use "UID" of data_time_conf and "countyFISP" from data_counties as our foreign key, and notice that the "countyFIPS" of data_counties is just UID of data_time_conf add 84000000
- So in the following codes, we adjust the keys such that they will match with each other

In [ ]:
data_time_conf['UID'] = data_time_conf['UID'].astype(int)
data_counties["countyFIPS"] = data_counties['countyFIPS'].astype(int)
data_counties['countyFIPS'] = data_counties['countyFIPS'] + 84000000

# Merge the two table
combined_conf = data_time_conf.merge(data_counties, left_on= 'UID', right_on = 'countyFIPS')

combined_conf.head(5)

#### Cleaning up NaN values

In [ ]:
combined_conf.isna().sum()

- Notice that there are lots of NaN values in some of the columns. 
- Here, in order to give a consistant and relatively accurate X matrix for fitting our models, we decided to drop out the columns with too many NaNs, since if there are too many NaNs in a category, it would be really hard to choose alternative for them since demographic information are very unique to each state, and filling up all the NaN data with "mean" "median" or "most frequent" will potentially bring misleading information to our models.
- To define "too many", we decided that if more than 20% of the data are NaN, we will drop it.

In [ ]:
combined_conf.loc[:, combined_conf.isnull().mean() > .2].isnull().mean()
combined_conf = combined_conf.loc[:, combined_conf.isnull().mean() <= .2]
combined_conf.loc[:, combined_conf.isnull().mean() > 0].isnull().mean()

Columns to Drop Direactly:
1. `State`: The Province_State from the time serise table has no NaN values, so Provinc_State can already represent the names of all the State.
2. `lat`: The Lat from the time serise table has no NaN values, so it can present all the locations
3. `lon`: The Lon from the time serise table has no NaN values, so it can present all the locations
4. `entertainment/gym`: We think gym and entertainment might have long-term effect for human body, but it is not very related to the virus that is happend right now.

Columns to take the "Mean" value to fill NaNs:
1. `#EligibleforMedicare2018`: Since it has such a small NaN percentage and Medicare system is relatively well-developed in the U.S., we decided to fill the mean for the NaN of this column
2. `All the rate, ratio, and percentile`: Since rate and ratio has already scaled, we can safely apply rate and ratio to states and counties with different populations.
3. `All the MortalityAge`: Since all of them have such low NaN precentage, we decided to fill in Mean for them, as Mean will average out the effect of states with large population and small population. 

Columns to take the "0" value to fill NaNs:
1. `>50 gatherings`: since gatherings might have a huge impact on confirmed cases, we rather to do it more safely by filling in 0s for these features with NaNs.
2. `>500 gatherings`: since gatherings might have a huge impact on confirmed cases, we rather to do it more safely by filling in 0s for these features with NaNs.

In [ ]:
combined_conf = combined_conf.drop(columns = ['State', 'lat', 'lon', 'entertainment/gym'])
combined_conf['>50 gatherings'] = combined_conf['>50 gatherings'].fillna(0)
combined_conf['>500 gatherings'] = combined_conf['>500 gatherings'].fillna(0)
combined_conf = combined_conf.fillna(combined_conf.mean())
# Indicator that there are no more NaN left in our dataframe!
combined_conf.loc[:, combined_conf.isnull().mean() > 0].isnull().mean().sum()

### 3. Create the All-in-1 DataFrame for training

In [ ]:
all_in_1 = combined_conf.merge(general_state, left_on = 'Province_State', right_on='Province_State')

all_in_1.head(5)

- Notice there are many columns within these two dataframe with repeated names, which Pandas matigate through it with adding "_x" "_y" at the end of the column names
- So, I clean out the data further by dropping the columns with overlapping names

In [ ]:
drop_list = ['UID_x', 'iso2', 'iso3', "code3", "FIPS_x", 'Admin2', 
                'Province_State', "Country_Region_x", "Combined_Key",
               "StateName", 'countyFIPS', 'STATEFP', 'COUNTYFP', 'CountyName', 
                'POP_LATITUDE', 'POP_LONGITUDE', 'FIPS_y', "UID_y", 
                "ISO3", 'CensusRegionName', 'Country_Region_y', 'Last_Update', 'CensusDivisionName']

all_in_1 = all_in_1.drop(columns = drop_list)

## EDA & Feature Selection

In [ ]:
def mse(y_test, y_true):
    return np.mean((y_test - y_true) ** 2)

def expo_fit(y):
    x = np.arange(-y.shape[0], 0)
    return np.exp(np.polyfit(x, np.log(y), 1)[1])

### 1. Import the data that has already been cleaned and merged

In [ ]:
data = pd.read_csv("all_in_1.csv")
X, y = data.drop(columns=['4/18/20']), data['4/18/20']

### 2.Data Visualizations

#### Exponential Interpolation

In [ ]:
results = []
for j in range(86,89):
    result = []
    for i in range(j-12, j-2):
        y_mat = X.iloc[:,i:j].to_numpy()
        exp_predict = np.apply_along_axis(expo_fit, 1, y_mat)
        exp_predict[np.isnan(exp_predict)] = 0
        result.append(mse(exp_predict, X.iloc[:,j]))
    results.append(result)

# special handling for interpolating 4/18/20 data
result = []
for i in range(78, 88):
    y_mat = X.iloc[:,i:90].to_numpy()
    exp_predict = np.apply_along_axis(expo_fit, 1, y_mat)
    exp_predict[np.isnan(exp_predict)] = 0
    result.append(mse(exp_predict, y))

results.append(result)

In [ ]:
x = np.arange(11, 1, -1)
labels = ['4/15/20', '4/16/20', '4/17/20', '4/18/20']
plt.figure(figsize=(8, 4))
for i in range(4):
    plt.plot(x, results[i], label=labels[i])
plt.legend()
plt.ylabel("mse of interpolation")
plt.xlabel("# of days used in interpolation")
plt.title("Figure C: performance of exponential interpolation v.s. # of previous days used", fontsize=11);

Our first assumption follows from the **common exponential model for epidemic growth**. We take advantage of this and try to answer "which days in the time series can accurately predict the future" using exponential interpolation. More specifically, to generalize this assumption and reduce our search space, we **fit exponential curves for "n consecutive days earlier" ($n \in [2, 11]$).**

#### Distribution Observation

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
sns.distplot(y[y < np.percentile(y, 80)], kde=False)
plt.xlabel("Confirmed cases in 4/18/20")
plt.ylabel("Frequency")
plt.title("Figure A: distribution of confirmed cases for counties < 80 percentile", fontsize=15);

plt.subplot(1, 2, 2)
sns.scatterplot(x="4/18/20", y="PopulationEstimate2018", data=data)
plt.xlabel("Confirmed cases in 4/18/20")
plt.title("Figure B: distplot of confirmed cases v.s. population", fontsize=15);

- In figure A, we plot out the **distribution of confirmed cases for counties within lower 80 percentiles**. It shows how left-skewed the distribution is as **80\% of counties have less than 80 cases and more than $\frac{1}{3}$ of counties have less than 10**. 
- Figure B gives us a more complete view of the distribution of populations and confirmed cases, as we can see **the clusters of points near the origin and some "outlier-like" points far away**, which helps us greatly in the downstream model selection. We also didn't find a strong correlation between population size and the confirmed case. 

### 3. LassoCV Automated Feature Selection

In [ ]:
sfm = SelectFromModel(LassoCV(normalize=True)).fit(X, y)

feature_imp_df = pd.DataFrame({"col_name": X.columns, "imp": sfm.estimator_.coef_})
feature_imp_df.sort_values(by="imp", ascending=False)[:20]

#### Feature Correlation Visualization Heatmap

In [ ]:
lasso_features = ['dem_to_rep_ratio', 'PopulationDensityperSqMile2010', 'PopFmle25-292010', '4/18/20']
extra_t_features = ['PopulationEstimate2018', 'stay at home', 
                    'Mortality_Rate', 'Testing_Rate', 'People_Hospitalized', 
                    'Incident_Rate', 'People_Tested', 'StrokeMortality','PopFmle65-742010',
                    'PopMale75-842010', 'PopFmle75-842010', 
                    'PopFmle45-542010','PopMale55-592010', 'PopFmle55-592010', 'PopMale60-642010',
                    'PopFmle60-642010',
                    'PopMale75-842010', 'PopFmle75-842010',
                    '3-YrMortalityAge55-64Years2015-17',
                    '3-YrMortalityAge65-74Years2015-17',
                    '3-YrMortalityAge75-84Years2015-17', '3-YrMortalityAge85+Years2015-17',]
heat_features = lasso_features + extra_t_features

In [ ]:
heat_df = all_in_1[heat_features]

plt.figure(figsize=(10, 9))
plot = sns.heatmap(heat_df.corr()[['4/18/20']].sort_values(by=['4/18/20'],ascending=False),
            cmap="YlGnBu", linewidths=.5, annot=True, annot_kws={"size": 10})
plot.set_title('Top Correlations between Confirmed cases of 4/18/20 with other Features');

## Model

In [ ]:
features = ['4/15/20',
 '4/16/20',
 '4/17/20',
 'dem_to_rep_ratio',
 'public schools',
 'FracMale2017',
 'DiabetesPercentage',
 'People_Tested',
 'HeartDiseaseMortality',
 ]

### 1. Simple Regressors (LinearRegression v.s. DecisionTree vs. KNN Regressor)

#### Feature Augmentation Attempt using KNN Regressor

In [ ]:
def dist_metric(loc1, loc2):
    lat1, lon1, lat2, lon2  = loc1[0], loc1[1], loc2[0], loc2[1]
    p = 0.017453292519943295 # Pi / 180
    a = 0.5 - np.cos((lat2 - lat1) * p) / 2 + np.cos(lat1 * p) * np.cos(lat2 * p) * \
    (1 - np.cos((lon2 - lon1) * p)) / 2
    return 12742 * np.arcsin(a ** 2) # 12742 = 2 * R

In [ ]:
X_KNN, y_KNN = data.loc[:, ['Lat_x', 'Long_']], data['4/18/20']

neigh = KNeighborsRegressor(n_neighbors=2, weights = 'distance', metric = dist_metric)
neigh.fit(X_KNN, y_KNN)

In [ ]:
# setting up a dictionary with {county_index: list of its K Nearest Neighbours} Pairings
clusters = neigh.kneighbors()[1]
neighbour_dict = {i: clusters[i] for i in X.index}

In [ ]:
# ref_pop is the poplation of the county being referenced to, neigh_pop is the population of one of its 
# neighbor counties counts, neigh_pops are np arrays, ref_pop is is an integer
def normalize_by_pop(stat_mat, neigh_pops, ref_pop):
    factors = [ref_pop/n_pop for n_pop in neigh_pops]
    weighted_sums = stat_mat @ factors
    return list(weighted_sums / len(neigh_pops))
 

f = ['4/15/20','4/16/20','4/17/20']
f_new = ['4/15/20_neigh_nor','4/16/20_neigh_nor','4/17/20_neigh_nor']

new_feature_list = []

for i in X.index:
    neigh_id = neighbour_dict.get(i)

    ref_pop = X.loc[i, 'PopulationEstimate2018']
    neigh_pops = X.loc[neigh_id, 'PopulationEstimate2018']
    stat_matrix = np.transpose(X.loc[neigh_id, f]) # a len(f) by num_neigh matrix
    
    new_feature_list.append(normalize_by_pop(stat_matrix, neigh_pops, ref_pop))

new_feature_df = pd.DataFrame(np.array(new_feature_list), columns=f_new)

X_new = pd.concat([X, new_feature_df], axis=1)

#### Train-test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[features], y, test_size=0.1, random_state=42)

X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(X_new[features + f_new], y, test_size=0.1, random_state=42)


#### Overall MSE of LinearRegressor and TreeRegresso

In [ ]:
linear_regr = LinearRegression(normalize=True)
linear_regr.fit(X_train, y_train)

tree_regr = DecisionTreeRegressor(max_depth=20, random_state=42)
tree_regr.fit(X_train, y_train)

linear_regr_KNN = LinearRegression(normalize=True)
linear_regr_KNN.fit(X_train_aug, y_train_aug)

tree_regr_KNN = DecisionTreeRegressor(max_depth=20, random_state=42)
tree_regr_KNN.fit(X_train_aug, y_train_aug)


linear_pred = np.round(np.maximum(0, linear_regr.predict(X_test)))
linear_mse = mse(linear_pred, y_test)

tree_pred = np.round(tree_regr.predict(X_test))
tree_mse = mse(tree_pred, y_test)

KNN_linear_pred = np.round(np.maximum(0, linear_regr_KNN.predict(X_test_aug)))
KNN_linear_mse = mse(KNN_linear_pred, y_test_aug)

KNN_tree_pred = np.round(tree_regr_KNN.predict(X_test_aug))
KNN_tree_mse = mse(KNN_tree_pred, y_test_aug)


print(f"Linear mse: {linear_mse}, Tree mse: {tree_mse}")
print(f"Linear_KNN mse: {KNN_linear_mse}, Tree_KNN mse: {KNN_tree_mse}")

- We can see that that the KNN Algorithm act as a mediator for the algorithms. It improves the Desion Tree Model quite significantly, by introducing noise, nuetralizing overfitting.
- Our data, concluded from EDA, can be better modeled by a linear model. Thus, KNN adds noise to it.

#### Check the mse for prediction of counties with small/middle/large confirmed cases

In [ ]:
per_25, per_75 = np.percentile(y_test, 25), np.percentile(y_test, 75)

small_indices = np.argwhere(y_test < per_25).flatten()
middle_indices = np.argwhere((y_test >= per_25) & (y_test < per_75)).flatten()
big_indices = np.argwhere(y_test >= per_75).flatten()

for indices in [small_indices, middle_indices, big_indices]:
    a = y_test.to_numpy()[indices]
    
    #Linear
    b = linear_pred[indices]
    # Decision Tree
    c = tree_pred[indices]
    #Linear + KNN
    d = KNN_linear_pred[indices]
    # Decision Tree + KNN
    e = KNN_tree_pred[indices]
    
    print(f"Linear mse: {mse(a, b)}, Tree mse: {mse(a, c)}") 
    print(f"Linear_KNN mse: {mse(a, d)}, Tree_KNN mse: {mse(a, e)}")

- We can see the above tendency is more obvious data is bigger. This give a nice intuition for fine tuning the final model: applying KNN to the larger percentile of data might be benefitial.

### 2. Final Regressor

In [ ]:
class finalRegressor(object):
    def __init__(self, threshold=50, depth=11, forest=False, randstate=42):
        self.thres = threshold
        if forest:
            self.tree_regr = RandomForestRegressor(n_estimators=50, max_depth=depth, random_state=randstate)
        else:
            self.tree_regr = DecisionTreeRegressor(max_depth=depth, random_state=randstate)
        self.linear_regr = LinearRegression(normalize=True)
    
    def fit(self, X_train, y_train):
        self.tree_regr.fit(X_train, y_train)
        self.linear_regr.fit(X_train, y_train)
        self.thres_val = np.percentile(y_train, self.thres)
    
    def predict(self, X_test):
        self.linear_pred = np.maximum(self.linear_regr.predict(X_test), 0)
        self.tree_pred = self.tree_regr.predict(X_test)
        self.cond_vec = ((self.linear_pred + self.tree_pred) / 2) < self.thres_val
        return np.round(self.tree_pred * self.cond_vec + self.linear_pred * ~self.cond_vec)


##### Using test data to tune the max_depth for DecisionTree

In [ ]:
depth_results = []
small_indices = np.argwhere(y_test < 100).flatten()

for depth in range(3, 30):
    dt = DecisionTreeRegressor(max_depth=depth)
    dt.fit(X_train, y_train)
    a, b = dt.predict(X_test)[small_indices], y_test.to_numpy()[small_indices]
    depth_results.append(mse(a, b))

In [ ]:
optimal_max_depth = np.argmin(depth_results) + 3
optimal_max_depth

In [ ]:
plt.plot(depth_results)

##### Using 5-fold Cross Validation to tune the best threshold of our final predictor

In [ ]:
kf = KFold(n_splits=5)
best_thres, best_val = None, float('inf')

for thres in range(20, 100):
    cv_result = []
    for train_index, test_index in kf.split(X_train):
        final_regr = finalRegressor(thres)
        final_regr.fit(X_train.iloc[train_index], y_train.iloc[train_index])
        final_pred = final_regr.predict(X_train.iloc[test_index])
        cv_result.append(mse(final_pred, y_train.iloc[test_index]))
    if best_val > np.mean(cv_result):
        best_val = np.mean(cv_result)
        best_thres = thres

In [ ]:
final_predictor = finalRegressor(best_thres) 
final_predictor.fit(X_train, y_train)
best_mse = mse(y_test, final_predictor.predict(X_test))
print("Our integrated model, after fine-tuning, achieves the MSE of " + str(best_mse) + " on testing set")

================================================================================================================
# END PROJECT